# Encode spike timing/ISI to symbols

In [ ]:
import pandas as pd

from scipy import stats
from scipy.stats import mannwhitneyu as mw
from scipy.stats import sem
from scipy.stats import iqr
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from scipy.stats import wilcoxon

import random

import math

import EntropyHub as EH

import numpy as np

import time

from os import listdir
from os.path import isfile, join

import neo

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('spt_isi.csv') #open file generated with 1 NEX to CSV

In [ ]:
def toFixed(numObj, digits=0):
    return f"{numObj:.{digits}f}"


In [ ]:
# spikes in bins function


def sib(isi_list):
    bins_num = int(np.sum(isi_list) // (1.6 * np.min(isi_list)))
    spt_times = [0]
    for el in isi_list:
        spt_times.append(spt_times[-1] + float(el))
    res = np.histogram(spt_times, bins=bins_num)[0]
    while len(res[res > 1]) != 0:
        bins_num += 1
        res = np.histogram(spt_times, bins=bins_num)[0]
    return list(res), bins_num


In [ ]:
# ISI to mean ISI function


def isitm(isi_list):
    mean_isi = np.mean(isi_list)
    res = []
    for el in isi_list:
        if el >= mean_isi:
            res.append(1)
        else:
            res.append(0)
    return res


In [ ]:
# ISI to ISI function


def isitisi(isi_list, dig=4):
    res = []
    for i in range(len(isi_list) - 1):
        i_1 = float(toFixed(isi_list[i + 1], digits=dig))
        i_0 = float(toFixed(isi_list[i], digits=dig))
        if i_1 > i_0:
            res.append(3)
        elif i_1 == i_0:
            res.append(2)
        elif i_1 < i_0:
            res.append(1)
    return res


In [ ]:
# isi_list = df['isi_list'][0].strip('][').split(', ')
# isi_list = [float(j) for j in isi_list]
# assib(isi_list)

In [ ]:
sib_res_1 = []  # spiketrain histogram
sib_res_2 = []  # spiketrain histogram optimal bins
isitm_res = []
isitisi_res = []

for i in range(len(df)):
    print("Нейрон №", i + 1)
    isi_list = df["isi_list"][i].strip("][").split(", ")
    isi_list = [float(j) for j in isi_list]

    sib_res = sib(isi_list)
    sib_res_1.append(sib_res[0])
    sib_res_2.append(sib_res[1])

    isitm_res.append(isitm(isi_list))

    isitisi_res.append(isitisi(isi_list))


In [ ]:
df['sib']=sib_res_1
df['sib_bins']=sib_res_2
df['isitm']=isitm_res
df['isitisi']=isitisi_res
# df['assib_hist']=assib_res_1
# df['assib']=assib_res_2
df.to_csv('encoded_isi_v3.csv')